In [22]:
import numpy as np
import os 
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import NullLocator
import matplotlib.colors as colors
from scipy import spatial
import SimpleITK as sitk
from sklearn import metrics 
from skimage import morphology, measure
from tqdm import tqdm

In [85]:
def tumor_splitser(segmentation, remove_fist_component = True):
    '''
    takes in segmentation of multiple conected components and splits them over images with only one component 
    takes np.array as input
    returns np.array dims=(z,y,x,components)

    '''
    if isinstance(segmentation, sitk.Image):
        segmentation = sitk.GetArrayFromImage(segmentation)
    
    segmentation = measure.label(segmentation)
    # print('max',np.max(segmentation))
    out = np.empty((segmentation.shape[0], segmentation.shape[1], segmentation.shape[2], np.max(segmentation)))

    
    for i in range(1, np.max(segmentation)+1):
        # print('i',i)
        split = np.zeros_like(segmentation, dtype = np.bool_)
        split[np.where(segmentation == i)] = True
        out[:,:,:,i-1] = split
        
    return out

def run_dice(segmentation_folder, gt_folder):
    '''
    Runs dice score over all components in segmentation and gt
    returns list of non-zero dice scores
    '''
    teller = 0
    dice_scores = []
    # loop over all files in segmentation folder
    for i in tqdm(os.listdir(segmentation_folder)):
        
        if i.endswith('.nii'):
            # load images
            segmentation = sitk.ReadImage(os.path.join(segmentation_folder, i))
            gt = sitk.ReadImage(gt_folder+i[0:8]+'_0000.nii')
            
            # split connected components
            segmentation = tumor_splitser(segmentation)
            gt = tumor_splitser(gt)

            # loop over all components in segmentation
            for j in range(1,segmentation.shape[3]):    
                for k in range(0,gt.shape[3]):
                    
                    dice_score = metrics.f1_score(segmentation[:,:,:,j].flatten(), gt[:,:,:,k].flatten())
                
                    if dice_score != 0:
                        dice_scores.append(dice_score)
                              
    return dice_scores

In [88]:
segmentation_folder = 'C:/Users/korat/Desktop/MEP/Data/WORC_CRLM/Pure_CRLM_data/predictions/'

gt_root = 'C:/Users/korat/Desktop/MEP/Data/nnUnet/pure_CRLM/task502_CRLM/'
gt_RAD  = 'C:/Users/korat/Desktop/MEP/Data/nnUnet/pure_CRLM/task502_CRLM/labelsTr/' #(RAD)
gt_STUD1 = gt_root + 'labelsTr_STUD1/'
gt_STUD2 = gt_root + 'labelsTr_STUD2/'
gt_CNN  = gt_root + 'labelsTr_CNN/'
gt_PhD  = gt_root + 'labelsTr_PhD/'

gt_RAD_dice   = run_dice(segmentation_folder, gt_RAD)
gt_STUD1_dice = run_dice(segmentation_folder, gt_STUD1)
gt_STUD2_dice = run_dice(segmentation_folder, gt_STUD2)
gt_CNN_dice = run_dice(segmentation_folder, gt_CNN)
gt_PhD_dice = run_dice(segmentation_folder, gt_PhD)

  0%|          | 0/34 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [87]:
root = 'C:/Users/korat/Desktop/MEP/Data/WORC_CRLM/Pure_CRLM_data/nnUNet_dice_reults/'

np.save(root + 'dice_scores.npy', gt_RAD_dice)
np.save(root + 'gt_STUD1_dice.npy', gt_STUD1_dice)
np.save(root + 'gt_STUD2_dice.npy', gt_STUD2_dice)
np.save(root + 'gt_CNN_dice.npy', gt_CNN_dice)
np.save(root + 'gt_PhD_dice.npy', gt_PhD_dice)